In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import os, sys, pyopencl
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt

#Finally, import our simulator
from SWESimulators import FBL, CTCS, DataOutput

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Finally, import our simulator
from SWESimulators import CTCS2Layer, DataOutput, PlotHelper

In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

In [ ]:
def setupTestCaseBump(nx, ny, dx, dy):
    h1 = np.ones((ny+2, nx+2), dtype=np.float32, order='C') * 10;
    eta1 = np.zeros((ny+2, nx+2), dtype=np.float32, order='C');
    u1 = np.zeros((ny+2, nx+1), dtype=np.float32, order='C');
    v1 = np.zeros((ny+1, nx+2), dtype=np.float32, order='C');

    h2 = np.ones((ny+2, nx+2), dtype=np.float32, order='C') * 50;
    eta2 = np.zeros((ny+2, nx+2), dtype=np.float32, order='C');
    u2 = np.zeros((ny+2, nx+1), dtype=np.float32, order='C');
    v2 = np.zeros((ny+1, nx+2), dtype=np.float32, order='C');

    #Create bump in center of domain for testing
    x_center = dx*nx/2.0
    y_center = dy*ny/2.0
    for j in range(-1, ny+1):
        for i in range(-1, nx+1):
            x = dx*i - x_center 
            y = dy*j - y_center
            size = 5000000.0*min(dx, dy)
            if (np.sqrt(x**2 + y**2) < size):
                ratio = h1[j+1, i+1] / (h1[j+1, i+1] + h2[j+1, i+1])
                eta1[j+1, i+1] =  np.exp(-(x**2/size+y**2/size))
                eta2[j+1, i+1] =  np.exp(-(x**2/size+y**2/size))
                #eta2[j+1, i+1] = np.exp(-(x**2/size+y**2/size))
    
    return h1,eta1,u1,v1,h2,eta2,u2,v2

In [ ]:
def setupTestCaseBumpInOneLayer(nx, ny, dx, dy):
    h1 = np.ones((ny+2, nx+2), dtype=np.float32, order='C') * 100;
    eta1 = np.zeros((ny+2, nx+2), dtype=np.float32, order='C');
    u1 = np.zeros((ny+2, nx+1), dtype=np.float32, order='C');
    v1 = np.zeros((ny+1, nx+2), dtype=np.float32, order='C');

    h2 = np.ones((ny+2, nx+2), dtype=np.float32, order='C') * 800;
    eta2 = np.zeros((ny+2, nx+2), dtype=np.float32, order='C');
    u2 = np.zeros((ny+2, nx+1), dtype=np.float32, order='C');
    v2 = np.zeros((ny+1, nx+2), dtype=np.float32, order='C');

    #Create bump in center of domain for testing
    x_center = dx*nx/2.0
    y_center = dy*ny/2.0
    for j in range(-1, ny+1):
        for i in range(-1, nx+1):
            x = dx*i - x_center 
            y = dy*j - y_center
            size = 5000000.0*min(dx, dy)
            if (np.sqrt(x**2 + y**2) < size):
                eta2[j+1, i+1] =  -np.exp(-(x**2/size+y**2/size)) * 50
    
    return h1,eta1,u1,v1,h2,eta2,u2,v2

In [ ]:
width = 250000
height = 350000

nx = 512
ny = 1024

dx = width/nx
dy = height/ny

#dt = 5 # setupTestCaseBump
dt = 1 # setupTestCaseBumpInOneLayer
g = 9.81
f = 0.00012 #0.0

rho1 = 1025.0
rho2 = 1030.0

r1 = 0.001
r2 = 0.0 #0.002

A = 1

#h1,eta1,u1,v1,h2,eta2,u2,v2 = setupTestCaseBump(nx, ny, dx, dy)
h1,eta1,u1,v1,h2,eta2,u2,v2 = setupTestCaseBumpInOneLayer(nx, ny, dx, dy)

#Initialize simulator
reload(CTCS2Layer)
sim = CTCS2Layer.CTCS2Layer(cl_ctx,\
                h1, eta1, u1, v1, \
                h2, eta2, u2, v2, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r1, r2, A, \
                rho1, rho2)


#Calculate radius from center of bump for plotting
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_center = dx*nx/2.0
y_center = dy*ny/2.0
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

fig = plt.figure()
reload(PlotHelper)
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, \
                                eta1[1:-1, 1:-1],\
                                u1[1:-1, :] + u2[1:-1, :], \
                                v1[:, 1:-1] + v2[:, 1:-1], \
                                eta2[1:-1, 1:-1], \
                                u2[1:-1, :], \
                                v2[:, 1:-1])

def animate(i, writer):
    if (i>0):
        t = sim.step(10.0)
    else:
        t = 0.0
    eta1, u1, v1, eta2, u2, v2 = sim.download()

    plotter.plot(eta1[1:-1, 1:-1],\
                 u1[1:-1, :] + u2[1:-1, :], \
                 v1[:, 1:-1] + v2[:, 1:-1], \
                 eta2[1:-1, 1:-1],\
                 u2[1:-1, :], \
                 v2[:, 1:-1])
    fig.suptitle("CTCS-2 layer Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / 50.0) + " % => t=" + str(t)
        fig.savefig("{:010.0f}_ctcs2l.png".format(t))
    
#anim = animation.FuncAnimation(fig, animate, range(50), interval=100)
#plt.close(anim._fig)
#anim

reload(DataOutput)
with DataOutput.CTCSNetCDFWriter("2LayerTest.nc", nx, ny, dx, dy, num_layers=1, ignore_ghostcells=True, \
                                 width=width, height=height) as writer:
    for i in range(30):
        if (i>0):
            t = sim.step(360.0)
        else:
            t = 0.0
        eta1, u1, v1, eta2, u2, v2 = sim.download()

        print "i: " + str(i) + " t: " + str(t)
        writer.write(i, t, eta1, u1/10, v1/10) # setupTestCaseBump
        #writer.write(i, t, eta1, u1/1000, v1/1000) # setupTestCaseBumpInOneLayer